# DevOps

I will be using the 128x128 CNN model for my DevOps project. To make it easier to quickly glance the important information I have made this section. 

### Model Training

Since we are done finding the best model and will now be using it for a production use case, I think it is better to train the model on the whole dataset rather than a smaller portion. 

In [16]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time

# ML
import tensorflow as tf
from tensorflow.keras.layers import *

# Miscellaneous
import warnings
warnings.filterwarnings("ignore")

# ML
import tensorflow as tf
import tensorflow.keras.backend as K

# Modelling
from tensorflow.keras.layers import *

tf.__version__

'2.15.0'

In [2]:
ROOT = "./Vegetable Images"
CLASSES = ['Broccoli','Capsicum','Bottle_Gourd','Radish','Tomato','Brinjal','Pumpkin','Carrot','Papaya','Cabbage','Bitter_Gourd','Cauliflower','Bean','Cucumber','Potato']
tfClasses = tf.constant(CLASSES)
TRAIN = tf.data.Dataset.list_files(f"{ROOT}/train/*/*")
TEST = tf.data.Dataset.list_files(f"{ROOT}/test/*/*")
VAL = tf.data.Dataset.list_files(f"{ROOT}/validation/*/*")
dataCats = ["train","test","validation"]

2024-01-19 10:20:31.688491: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-01-19 10:20:31.688519: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-01-19 10:20:31.688527: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-01-19 10:20:31.688605: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-19 10:20:31.688655: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
classCounts = []
for cat in dataCats:
    class_distro = [len(tf.data.Dataset.list_files(f"{ROOT}/{cat}/{i}/*")) for i in CLASSES]
    classCounts.append(class_distro)
    
train_distro = classCounts[0]

targetSize = max(train_distro)
additional_needed = [targetSize-i for i in train_distro]

aug_train = []
for i,v in enumerate(CLASSES):
    path = f'{ROOT}/train/{v}'
    imgNeeded = additional_needed[i]
    images = os.listdir(path)[:imgNeeded]
    aug_train.extend([path + "/" + i for i in images])
train_data_aug = tf.data.Dataset.from_tensor_slices(aug_train)

def createPreprocessor(imgSize):
    def processing(path):
        label = tf.strings.split(path , os.path.sep)
        one_hot = label[-2] == tfClasses
        label = tf.argmax(one_hot)
        label = tf.one_hot(label, depth=len(CLASSES))
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [imgSize, imgSize])
        img = tf.image.rgb_to_grayscale(img)
        img = img / 255.0
        return img , label
    return processing
    
def augmentation(image,label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.rot90(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    return image , label

def configure_for_performance(ds):
  ds = ds.cache()
  ds = ds.shuffle(buffer_size=1000)
  ds = ds.batch(32)
  ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
  return ds

train_ori_large = TRAIN.map(createPreprocessor(128))
train_aug_large = TRAIN.map(createPreprocessor(128)).map(augmentation)
train_large_ds = train_ori_large.concatenate(train_aug_large)

test_large_ds = TEST.map(createPreprocessor(128))
val_large_ds = VAL.map(createPreprocessor(128))

data = train_large_ds.concatenate(test_large_ds).concatenate(val_large_ds)
data = configure_for_performance(data)

train_ori_small = TRAIN.map(createPreprocessor(31))
train_aug_small = TRAIN.map(createPreprocessor(31)).map(augmentation)
train_small_ds = train_ori_small.concatenate(train_aug_small)

test_small_ds = TEST.map(createPreprocessor(31))
val_small_ds = VAL.map(createPreprocessor(31))

data_small = train_small_ds.concatenate(test_small_ds).concatenate(val_small_ds)
data_small = configure_for_performance(data_small)

In [4]:
reg_cnn_model = tf.keras.Sequential([
    Conv2D(16, 3, activation='relu', input_shape=(31, 31, 1)),
    BatchNormalization(),
    MaxPooling2D(),
    
    Conv2D(32, 3, activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),
    
    Conv2D(64, 3, activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),
    
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5), 
    Dense(15, activation='softmax')
])

In [11]:
reg_cnn_model_large = tf.keras.Sequential([
    Conv2D(32, 3, activation='relu', input_shape=(128, 128, 1)),
    BatchNormalization(),
    MaxPooling2D(),
    
    Conv2D(64, 3, activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),
    
    Conv2D(128, 3, activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),
    
    Conv2D(256, 3, activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),
    
    Flatten(),
    
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5), 
    
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5), 
    
    Dense(15, activation='softmax')
])

In [7]:
es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=1, min_lr=1e-6)

In [8]:
reg_cnn_model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
modelHist = reg_cnn_model.fit(
      data_small, epochs=25, callbacks=[es,reduce_lr]
    )

reg_cnn_model.save("./cnn/small", save_format="tf")

Epoch 1/25


2024-01-19 10:25:19.314150: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


752/752 [==============================] - 18s 20ms/step - loss: 3.1853 - accuracy: 0.2097 - lr: 0.0010
Epoch 2/25
752/752 [==============================] - 13s 17ms/step - loss: 2.1016 - accuracy: 0.3494 - lr: 0.0010
Epoch 3/25
752/752 [==============================] - 13s 17ms/step - loss: 1.8340 - accuracy: 0.4186 - lr: 0.0010
Epoch 4/25
752/752 [==============================] - 13s 17ms/step - loss: 1.6516 - accuracy: 0.4746 - lr: 0.0010
Epoch 5/25
752/752 [==============================] - 13s 17ms/step - loss: 1.5460 - accuracy: 0.5139 - lr: 0.0010
Epoch 6/25
752/752 [==============================] - 13s 17ms/step - loss: 1.4566 - accuracy: 0.5486 - lr: 0.0010
Epoch 7/25
752/752 [==============================] - 13s 17ms/step - loss: 1.3730 - accuracy: 0.5733 - lr: 0.0010
Epoch 8/25
752/752 [==============================] - 13s 17ms/step - loss: 1.3249 - accuracy: 0.5942 - lr: 0.0010
Epoch 9/25
752/752 [==============================] - 13s 17ms/step - loss: 1.2784 - accura

INFO:tensorflow:Assets written to: ./cnn/small/assets


In [12]:
reg_cnn_model_large.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
modelHist2 = reg_cnn_model_large.fit(
      data, epochs=15, callbacks=[es,reduce_lr]
    )

reg_cnn_model_large.save("./cnn/large", save_format="tf")

Epoch 1/15


752/752 [==============================] - 37s 47ms/step - loss: 1.6099 - accuracy: 0.5079 - lr: 0.0010
Epoch 2/15
752/752 [==============================] - 36s 48ms/step - loss: 0.7013 - accuracy: 0.7878 - lr: 0.0010
Epoch 3/15
752/752 [==============================] - 36s 47ms/step - loss: 0.3771 - accuracy: 0.8877 - lr: 0.0010
Epoch 4/15
752/752 [==============================] - 36s 47ms/step - loss: 0.2551 - accuracy: 0.9231 - lr: 0.0010
Epoch 5/15
752/752 [==============================] - 35s 47ms/step - loss: 0.1957 - accuracy: 0.9390 - lr: 0.0010
Epoch 6/15
752/752 [==============================] - 36s 48ms/step - loss: 0.1503 - accuracy: 0.9539 - lr: 0.0010
Epoch 7/15
752/752 [==============================] - 36s 47ms/step - loss: 0.1055 - accuracy: 0.9670 - lr: 0.0010
Epoch 8/15
752/752 [==============================] - 35s 47ms/step - loss: 0.1128 - accuracy: 0.9645 - lr: 0.0010
Epoch 9/15
752/752 [==============================] - 37s 49ms/step - loss: 0.0568 - accura

INFO:tensorflow:Assets written to: ./cnn/large/assets


Some Useful Information about our models.

In [13]:
reg_cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 29, 29, 16)        160       
                                                                 
 batch_normalization (Batch  (None, 29, 29, 16)        64        
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 12, 12, 32)        4640      
                                                                 
 batch_normalization_1 (Bat  (None, 12, 12, 32)        128       
 chNormalization)                                                
                                                        

In [14]:
reg_cnn_model_large.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 126, 126, 32)      320       
                                                                 
 batch_normalization_9 (Bat  (None, 126, 126, 32)      128       
 chNormalization)                                                
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 batch_normalization_10 (Ba  (None, 61, 61, 64)        256       
 tchNormalization)                                               
                                                      

In [17]:
# Measure prediction time for final_31
start_time = time.time()
prediction_31 = reg_cnn_model.evaluate(configure_for_performance(test_small_ds))
end_time = time.time()
time_taken_31 = end_time - start_time

# Measure prediction time for final_128
start_time = time.time()
prediction_128 = reg_cnn_model_large.evaluate(configure_for_performance(test_large_ds))
end_time = time.time()
time_taken_128 = end_time - start_time

# Print the prediction times
print(f"Prediction time for final_31: {time_taken_31} seconds")
print(f"Prediction time for final_128: {time_taken_128} seconds")

94/94 [==============================] - 2s 20ms/step - loss: 0.1092 - accuracy: 0.9597
Prediction time for final_31: 1.6732401847839355 seconds
Prediction time for final_128: 2.2491331100463867 seconds


As you can see the larger model takes abit longer but is also abit more accurate.

## Model Serving

Now that we are done training and saving our models in the TF SavedModel format, we can run a series of commands as seen below to create our Tensorflow  Serving Model.